In [1]:
#from google.colab import drive
#drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


import os 
os.getcwd()
#path = os.join(os.getcwd(), '/drive/app')

path = r'/content/drive/My Drive/app'
os.chdir(path)
print(os.getcwd())
print(os.listdir())

In [27]:
import funciones_wgan as f
import numpy as np
#import cv2
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K
import matplotlib.pyplot as plt
%matplotlib inline
from time import time
from keras.callbacks import TensorBoard
import tensorflow as tf

# Variables generales

In [3]:
EPOCHS = 9000
BATCH_SIZE = 8
# The training ratio is the number of discriminator updates
# per generator update. The paper uses 5.
TRAINING_RATIO = 1
GRADIENT_PENALTY_WEIGHT = 10  # As per the paper
INPUT_LEN = 128
output_dir = 'output/'
discriminator_weights = 'Weights/discriminator_epoch_950.h5'
generator_weights = 'Weights/generator_epoch_950.h5'
muestra = True # Si queremos coger una muestra de las imágenes. En true se cogen 140 imágenes.

In [4]:
def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()

In [5]:
try:
    initial_epoch = int(generator_weights.split('_')[2].split('.')[0]) + 1
except:
    initial_epoch = 0
final_epoch = initial_epoch + EPOCHS - 1

# Discriminador

In [6]:
def make_discriminator():
    """Creates a discriminator model that takes an image as input and outputs a single
    value, representing whether the input is real or generated. Unlike normal GANs, the
    output is not sigmoid and does not represent a probability! Instead, the output
    should be as large and negative as possible for generated inputs and as large and
    positive as possible for real inputs.
    Note that the improved WGAN paper suggests that BatchNormalization should not be
    used in the discriminator."""
    
    model = Sequential()
    model.add(Convolution2D(32, 5, padding='same', strides=[2, 2], input_shape=(256, 256, 3)))
    model.add(LeakyReLU())

    model.add(Convolution2D(64, 5, kernel_initializer='he_normal', strides=[2, 2], padding='same'))
    model.add(LeakyReLU())

    model.add(Convolution2D(128, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Convolution2D(256, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Convolution2D(512, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Convolution2D(1024, 5, kernel_initializer='he_normal', padding='same', strides=[2, 2]))
    model.add(LeakyReLU())

    model.add(Flatten())
    #model.add(Dense(1024 * 4 * 4, kernel_initializer='he_normal'))
    #model.add(LeakyReLU())
    model.add(Dense(1, kernel_initializer='he_normal'))
    
    return model

# Generador

In [7]:
def make_generator():
    """Creates a generator model that takes a 128-dimensional noise vector as a "seed",
    and outputs images of size 256x256x3."""
    model = Sequential()
    
    model.add(Dense(4 * 4 * 2048, input_dim=INPUT_LEN))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    model.add(Reshape((4, 4, 2048), input_shape=(4 * 4 * 2048,)))
    bn_axis = -1

    model.add(Conv2DTranspose(1024, 5, strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(512, 5, strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(256, 5, strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(128, 5, strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(64, 5, strides=2, padding='same'))
    model.add(BatchNormalization(axis=bn_axis))
    model.add(LeakyReLU())

    model.add(Conv2DTranspose(3, 5, strides=2, padding='same', activation='tanh'))
    # El output de esta última es 256x256x3
    
    return model

# Carga de datos

El archivo total_three_datasets_sorted.npy es una matriz que ya tiene las tres capas:
* 0: T1
* 1: FLAIR
* 2: Máscara

In [8]:
try:
    images = np.load('total_three_datasets_sorted.npy')
except:
    images = np.load('images_three_datasets_sorted.npy')
    masks = np.load('masks_three_datasets_sorted.npy')
    
    # Normalizado entre -1 y +1. Esto lo hace sobre toda la imagen, no sobre la capa T1 o FLAIR por separado.
    # No tengo muy claro que sea correcto
    images = [2.*(image - np.min(image))/np.ptp(image) - 1 for image in images]
        
    images = np.concatenate((images, masks), axis=3)
    
    # El generador toma imágenes 256x256x3. Como las tenemos 200x200, hay que redimensionarlas:
    dim_final = (256, 256)
    images = np.array([cv2.resize(image, dim_final, interpolation = cv2.INTER_AREA) for image in images])
    
    np.save('total_three_datasets_sorted.npy', images)
    del masks

In [9]:
images.shape

(2780, 256, 256, 3)

Si se trabaja con Tensorflow está bien porque los canales están en la última dimensión del array.

In [10]:
if muestra:
    images_shuff = images[:]
    np.random.shuffle(images_shuff)
    images_shuff = images_shuff[0:140,...]
    images = images_shuff[:]

In [11]:
print(np.amin(images[34,...,0]), np.amin(images[34,...,1]))

-0.85898185 -1.0


In [12]:
n_images = images.shape[0]
print(n_images)

140


# Redes

Inicialización de generador y discriminador.

In [13]:
generator = make_generator()
discriminator = make_discriminator()

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
try:
    generator.load_weights(generator_weights)
    discriminator.load_weights(discriminator_weights)
except:
    pass

### Generador

In [15]:
# The generator_model is used when we want to train the generator layers.
# As such, we ensure that the discriminator layers are not trainable.
# Note that once we compile this model, updating .trainable will have no effect within
# it. As such, it won't cause problems if we later set discriminator.trainable = True
# for the discriminator_model, as long as we compile the generator_model first.
for layer in discriminator.layers:
    layer.trainable = False
    
discriminator.trainable = False
generator_input = Input(shape=(INPUT_LEN,))
generator_layers = generator(generator_input)
discriminator_layers_for_generator = discriminator(generator_layers)
generator_model = Model(inputs=[generator_input], outputs=[discriminator_layers_for_generator])
# We use the Adam paramaters from Gulrajani et al.
generator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9), loss=f.wasserstein_loss)

### Discriminador

In [16]:
# Now that the generator_model is compiled, we can make the discriminator
# layers trainable.
for layer in discriminator.layers:
    layer.trainable = True
    
for layer in generator.layers:
    layer.trainable = False
    
discriminator.trainable = True
generator.trainable = False

# The discriminator_model is more complex. It takes both real image samples and random
# noise seeds as input. The noise seed is run through the generator model to get
# generated images. Both real and generated images are then run through the
# discriminator. Although we could concatenate the real and generated images into a
# single tensor, we don't (see model compilation for why).
real_samples = Input(shape=images.shape[1:])
generator_input_for_discriminator = Input(shape=(INPUT_LEN,))
generated_samples_for_discriminator = generator(generator_input_for_discriminator)
discriminator_output_from_generator = discriminator(generated_samples_for_discriminator)
discriminator_output_from_real_samples = discriminator(real_samples)

In [17]:
# We also need to generate weighted-averages of real and generated samples,
# to use for the gradient norm penalty.
averaged_samples = f.RandomWeightedAverage()([real_samples,
                                            generated_samples_for_discriminator])
# We then run these samples through the discriminator as well. Note that we never
# really use the discriminator output for these samples - we're only running them to
# get the gradient norm for the gradient penalty loss.
averaged_samples_out = discriminator(averaged_samples)

In [18]:
# The gradient penalty loss function requires the input averaged samples to get
# gradients. However, Keras loss functions can only have two arguments, y_true and
# y_pred. We get around this by making a partial() of the function with the averaged
# samples here.
partial_gp_loss = f.partial(f.gradient_penalty_loss,
                            averaged_samples=averaged_samples, gradient_penalty_weight=GRADIENT_PENALTY_WEIGHT)
# Functions need names or Keras will throw an error
partial_gp_loss.__name__ = 'gradient_penalty'

In [19]:
# Keras requires that inputs and outputs have the same number of samples. This is why
# we didn't concatenate the real samples and generated samples before passing them to
# the discriminator: If we had, it would create an output with 2 * BATCH_SIZE samples,
# while the output of the "averaged" samples for gradient penalty
# would have only BATCH_SIZE samples.

# If we don't concatenate the real and generated samples, however, we get three
# outputs: One of the generated samples, one of the real samples, and one of the
# averaged samples, all of size BATCH_SIZE. This works neatly!
discriminator_model = Model(inputs=[real_samples, generator_input_for_discriminator],
                            outputs=[discriminator_output_from_real_samples, discriminator_output_from_generator, averaged_samples_out])

In [20]:
# We use the Adam paramaters from Gulrajani et al. We use the Wasserstein loss for both
# the real and generated samples, and the gradient penalty loss for the averaged samples
discriminator_model.compile(optimizer=Adam(0.0001, beta_1=0.5, beta_2=0.9),
                            loss=[f.wasserstein_loss, f.wasserstein_loss, partial_gp_loss])

# We make three label vectors for training. positive_y is the label vector for real
# samples, with value 1. negative_y is the label vector for generated samples, with
# value -1. The dummy_y vector is passed to the gradient_penalty loss function and
# is not used.
positive_y = np.ones((BATCH_SIZE, 1), dtype=np.float32)
negative_y = -positive_y
dummy_y = np.zeros((BATCH_SIZE, 1), dtype=np.float32)

In [24]:
log_path = './logs'
if not os.isdir(log_path):
    os.mkdir(log_path)
callback = TensorBoard(log_path)
callback.set_model(generator_model)

In [28]:
def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()

In [ ]:
intervado_guardado = 50

for epoch in range(initial_epoch, final_epoch):
    start = time()
    np.random.shuffle(images)
    print("Epoch: ", epoch)
    print("Number of batches: ", int(n_images // BATCH_SIZE))
    discriminator_loss = []
    generator_loss = []
    minibatches_size = BATCH_SIZE * TRAINING_RATIO
    print('Tenemos ', int(n_images // (BATCH_SIZE * TRAINING_RATIO)), ' minibatches.')
    for i in range(int(n_images // (BATCH_SIZE * TRAINING_RATIO))):
        discriminator_minibatches = images[i * minibatches_size: (i + 1) * minibatches_size]
        
        for j in range(TRAINING_RATIO):
            image_batch = discriminator_minibatches[j * BATCH_SIZE: (j + 1) * BATCH_SIZE]
            noise = np.random.normal(0, 1, (BATCH_SIZE, INPUT_LEN)).astype(np.float32)
            #noise = np.random.uniform(-1,1,(BATCH_SIZE, INPUT_LEN)).astype(np.float32)
            discriminator_loss_val = discriminator_model.train_on_batch([image_batch, noise], [positive_y, negative_y, dummy_y])
            discriminator_loss.append(discriminator_loss_val)
        
        #generator_loss_val = generator_model.train_on_batch(np.random.uniform(-1,1,(BATCH_SIZE, INPUT_LEN)), positive_y)
        generator_loss_val = generator_model.train_on_batch(np.random.normal(0, 1, (BATCH_SIZE, INPUT_LEN)), positive_y)
        generator_loss.append(generator_loss_val)
        
    
    print(f'Epoch {epoch} took {time() - start}')
    write_log(callback, ['g_loss'], [generator_loss_val], epoch)
    
    if epoch % intervado_guardado == 0:
        print('Saving weights')
        generator.save_weights(f'Weights/generator_epoch_{epoch}.h5')
        discriminator.save_weights(f'Weights/discriminator_epoch_{epoch}.h5')
        f.sample_best_images(generator, discriminator, output_dir, epoch, 10)
        try:
            os.remove(f'/content/drive/My Drive/app/Weights/discriminator_epoch_{epoch-intervado_guardado}.h5')
        except:
            pass
        try:
            os.remove(f'/content/drive/My Drive/app/Weights/generator_epoch_{epoch-intervado_guardado}.h5')
        except:
            pass
        #f.generate_images(generator, output_dir, epoch, 10, method='FLAIR')
        #f.generate_images(generator, output_dir, epoch, 10, method='T1')

Epoch:  951
Number of batches:  17
Tenemos  17  minibatches.
Epoch 951 took 143.32084202766418
Epoch:  952
Number of batches:  17
Tenemos  17  minibatches.
Epoch 952 took 139.83846163749695
Epoch:  953
Number of batches:  17
Tenemos  17  minibatches.
Epoch 953 took 140.57446932792664
Epoch:  954
Number of batches:  17
Tenemos  17  minibatches.
Epoch 954 took 143.28716254234314
Epoch:  955
Number of batches:  17
Tenemos  17  minibatches.
Epoch 955 took 147.33034348487854
Epoch:  956
Number of batches:  17
Tenemos  17  minibatches.
Epoch 956 took 144.62658524513245
Epoch:  957
Number of batches:  17
Tenemos  17  minibatches.
Epoch 957 took 147.58366703987122
Epoch:  958
Number of batches:  17
Tenemos  17  minibatches.
Epoch 958 took 144.48497223854065
Epoch:  959
Number of batches:  17
Tenemos  17  minibatches.
Epoch 959 took 142.63991618156433
Epoch:  960
Number of batches:  17
Tenemos  17  minibatches.
Epoch 960 took 142.84534215927124
Epoch:  961
Number of batches:  17
Tenemos  17  mi